In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [42]:
# 创建一个从word->index的字典
word_to_index = {"hello":0, "word":1}
# 神经网络的embedding layer
embeds = nn.Embedding(2, 5)

# 测试代码 通过上述的embedding layer 查找 hello
lookup_tensor = torch.tensor([word_to_index["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


## 训练embedding

In [48]:
# CONTEXT_SIZE 上下文的范围
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# 用来测试的句子
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

ngrams = [
        (
            [test_sentence[i-j-1] for j in range(CONTEXT_SIZE)],
            test_sentence[i]
        )
        for i in range(CONTEXT_SIZE, len(test_sentence))
]

vocab = set(test_sentence)
word_to_index = {word:i for i, word in enumerate(vocab)}

In [70]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim ,128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        print("inputs: ", inputs.shape)
        embeds = self.embeddings(inputs).view((1, -1))
        print("embeds: ", embeds.shape)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        print("out: ", out.shape)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [71]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# fit
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for context, target in ngrams:
        context_index = torch.tensor([word_to_index[w] for w in context], dtype=torch.long)
        
        model.zero_grad()
        log_probs = model(context_index)
        loss = loss_function(log_probs, torch.tensor([word_to_index[target]], dtype=torch.long))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    losses.append(total_loss)

# 打印出损失函数的值
print(losses)

# 打印出来beauty的词向量
print(model.embeddings.weight[word_to_index["beauty"]])


inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.

inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.Size([1, 20])
out:  torch.Size([1, 97])
inputs:  torch.Size([2])
embeds:  torch.